In [73]:
import requests

year = 2023
day = 21

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open("cookie.txt", "r") as file:
    session = file.read()

headers = {"cookie": f"session={session}"}

with open(f"inputs/day_{str(day).zfill(2)}_input.txt", "wb") as file:
    file.write(requests.get(url, headers=headers).content)
    
aoc = ''.join(open(f"inputs/day_{str(day).zfill(2)}_input.txt", "r"))

def use_input(input_string, debug=False):
    source = [x.strip() for x in input_string.split("\n")]
    if debug:
        print(source)
    G = [[x for x in line] for line in source]
    R = len(G)
    C = len(G[0])
    if debug:
        print(G[:2][:2], R,C)
    return lines, G, R, C
    
lines, G, R, C = use_input(aoc)

In [74]:
test = """...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
..........."""

lines, G, R, C = use_input(aoc)

found = False

for r, row in enumerate(G):
    for c, col in enumerate(row):
        if col == 'S':
            s_r = r
            s_c = c
            G[s_r][s_c] = '.'
            found = True
            break
    if found:
        break
        
print(s_r, s_c)

def take_step(t_r, t_c):
    positions = []
    for d in [[0, 1], [1, 0], [-1, 0], [0, -1]]:
        n_r = t_r + d[0]
        n_c = t_c + d[1]
        if 0 <= n_r < R and 0 <= n_c < C and G[n_r][n_c] == '.':
            positions.append((n_r,n_c))
    return tuple(positions)

visited = {(s_r,s_c)}
for i in range(64):
    n_visited = set({})
    for step in visited:
        next_steps = take_step(step[0], step[1])
        n_visited = n_visited.union(next_steps)
    visited = n_visited
    print("VIS", i, len(visited))

65 65
VIS 0 4
VIS 1 7
VIS 2 14
VIS 3 22
VIS 4 33
VIS 5 43
VIS 6 57
VIS 7 69
VIS 8 87
VIS 9 104
VIS 10 125
VIS 11 146
VIS 12 170
VIS 13 191
VIS 14 223
VIS 15 248
VIS 16 277
VIS 17 304
VIS 18 346
VIS 19 378
VIS 20 417
VIS 21 458
VIS 22 494
VIS 23 531
VIS 24 573
VIS 25 621
VIS 26 663
VIS 27 721
VIS 28 764
VIS 29 824
VIS 30 870
VIS 31 941
VIS 32 985
VIS 33 1065
VIS 34 1101
VIS 35 1190
VIS 36 1232
VIS 37 1321
VIS 38 1366
VIS 39 1460
VIS 40 1502
VIS 41 1596
VIS 42 1648
VIS 43 1734
VIS 44 1801
VIS 45 1889
VIS 46 1953
VIS 47 2048
VIS 48 2117
VIS 49 2214
VIS 50 2285
VIS 51 2391
VIS 52 2461
VIS 53 2581
VIS 54 2649
VIS 55 2762
VIS 56 2833
VIS 57 2947
VIS 58 3036
VIS 59 3155
VIS 60 3234
VIS 61 3401
VIS 62 3489
VIS 63 3666


In [88]:
test = """...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
..........."""

lines, G, R, C = use_input(test)

found = False

for r, row in enumerate(G):
    for c, col in enumerate(row):
        if col == 'S':
            s_r = r
            s_c = c
            G[s_r][s_c] = '.'
            found = True
            break
    if found:
        break
        
print(s_r, s_c)

def take_step(t_r, t_c, t_m_r, t_m_c):
    positions = []
    for d in [[0, 1], [1, 0], [-1, 0], [0, -1]]:
        n_r = t_r + d[0]
        n_c = t_c + d[1]
        if 0 <= n_r < R and 0 <= n_c < C and G[n_r][n_c] == '.':
            positions.append((n_r,n_c, t_m_r, t_m_c))
        if n_r == -1 :
            n_r = R -1
            if G[n_r][n_c] == '.':
                positions.append((n_r,n_c, t_m_r - 1, t_m_c))
        if n_c == -1:
            n_c = C - 1
            if G[n_r][n_c] == '.':
                positions.append((n_r,n_c, t_m_r, t_m_c - 1))
        if n_r == R:
            n_r = 0
            if G[n_r][n_c] == '.':
                positions.append((n_r,n_c, t_m_r + 1, t_m_c))
        if n_c == C:
            n_c = 0
            if G[n_r][n_c] == '.':
                positions.append((n_r,n_c, t_m_r, t_m_c + 1))                
    
    return tuple(positions)

visited = {(s_r,s_c, 0, 0)}
for i in range(10000):
    n_visited = set({})
    for step in visited:
        next_steps = take_step(step[0], step[1], step[2], step[3])
        n_visited = n_visited.union(next_steps)
    visited = n_visited
    if i+1 in [6,10,50,100,500,1000,5000]:
        print("VIS", i, len(visited))

5 5
VIS 5 16
VIS 9 50
VIS 49 1594
VIS 99 6536


KeyboardInterrupt: 

In [97]:
test = """...........
.....###.#.
.###.##..#.
..#.#...#..
....#.#....
.##..S####.
.##..#...#.
.......##..
.##.#.####.
.##..##.##.
..........."""

lines, G, R, C = use_input(test)

found = False

for r, row in enumerate(G):
    for c, col in enumerate(row):
        if col == 'S':
            s_r = r
            s_c = c
            G[s_r][s_c] = '.'
            found = True
            break
    if found:
        break
        
print(s_r, s_c)

def take_step(t_r, t_c,):
    positions = []
    for d in [[0, 1], [1, 0], [-1, 0], [0, -1]]:
        n_r = t_r + d[0]
        n_c = t_c + d[1]
        if 0 <= n_r < R and 0 <= n_c < C and G[n_r][n_c] == '.':
            positions.append((n_r,n_c, 0, 0))
        if n_r == -1 :
            n_r = R -1
            if G[n_r][n_c] == '.':
                positions.append((n_r,n_c, - 1, 0))
        if n_c == -1:
            n_c = C - 1
            if G[n_r][n_c] == '.':
                positions.append((n_r,n_c, 0, - 1))
        if n_r == R:
            n_r = 0
            if G[n_r][n_c] == '.':
                positions.append((n_r,n_c,  1, 0))
        if n_c == C:
            n_c = 0
            if G[n_r][n_c] == '.':
                positions.append((n_r,n_c, 0, 1))                
    
    return tuple(positions)

def take_all_steps(i_visited):
    n_visited = set({})
    for step in i_visited:
        next_steps = take_step(step[0], step[1])
        n_visited = n_visited.union(next_steps)
    return n_visited

visited = {(0,0): {(s_r, s_c)}}
for i in range(6):
    n_visited = take_all_steps(visited[(0,0)])
    #print(n_visited)
    visited[(0,0)] = n_visited
    #if i+1 in [6,10,50,100,500,1000,5000]:
    print("VIS", i, len(visited[(0,0)]))
        
# instead consider a frame, and how that maps to another frame.

5 5
VIS 0 2
VIS 1 4
VIS 2 6
VIS 3 9
VIS 4 13
VIS 5 16
